### vector db 활용실습

- 야놀자의 제주도 호텔 리뷰를 바탕으로 사용자가 원하는 호텔의 종류를 입력받고 사용자의 키워드와 높은 유사도를 가진 리뷰를 검색하여 호텔 추천

In [15]:
import pandas as pd
import chromadb
import numpy as np
from sentence_transformers import SentenceTransformer


In [ ]:
df = pd.read_csv('./data/jeju_hotel_review_raw.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12920 entries, 0 to 12919
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    12920 non-null  int64 
 1   hotel         12920 non-null  object
 2   star          12920 non-null  int64 
 3   review        12920 non-null  object
 4   clean_review  12920 non-null  object
 5   date          12920 non-null  object
dtypes: int64(2), object(4)
memory usage: 605.8+ KB


In [17]:
df['hotel'].nunique()

41

In [18]:
df['hotel'].value_counts()


hotel
아트스테이 서귀포 하버                 1270
봄그리고가을 호텔&리조트                 736
호텔 리젠트마린                      574
호텔위드 제주                       505
휘슬락 호텔                        492
호텔 시리우스 제주                    482
호텔샬롬 제주                       425
코업시티호텔 성산                     386
라마다 제주시티 호텔                   384
코델리아s 호텔 앤 리조트                357
브라운 스위트 제주                    348
베스트웨스턴 제주                     322
신화관 제주 신화월드 호텔앤리조트            307
더 베스트 제주 성산(구 라마다 앙코르 성산)     305
유탑유블레스호텔 제주함덕                 300
제주 헤이, 서귀포                    292
호텔 컬리넌 제주                     289
롯데 시티호텔 제주                    288
호텔 앨리스 앤 트렁크                  288
호텔 에어시티 제주                    286
랜딩관 제주신화월드 호텔앤리조트             275
윈스토리 호텔 서귀포                   271
호텔 골든데이지 서귀포 오션               262
에벤에셀 제주 함덕호텔                  259
제주 센트럴 시티 호텔                  257
호텔 케니 서귀포                     238
제주 부영호텔&리조트                   236
플레이스 캠프 제주                    219
팜파스호텔 제주                      213
더 포 그레이스

In [19]:
df_1_ = df.groupby('hotel').head(50).reset_index(drop=True)
df_1_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2050 entries, 0 to 2049
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    2050 non-null   int64 
 1   hotel         2050 non-null   object
 2   star          2050 non-null   int64 
 3   review        2050 non-null   object
 4   clean_review  2050 non-null   object
 5   date          2050 non-null   object
dtypes: int64(2), object(4)
memory usage: 96.2+ KB


* 임베딩 모델 로드

In [20]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

* cromadb 클라이언트 객체 생성

In [21]:
hotel_chroma_client = chromadb.PersistentClient(path="./hotel_chroma_db")
hotel_collection = hotel_chroma_client.get_or_create_collection(name="hotel_documents")


In [22]:
df_1_['hotel']

0       아트스테이 서귀포 하버
1       아트스테이 서귀포 하버
2       아트스테이 서귀포 하버
3       아트스테이 서귀포 하버
4       아트스테이 서귀포 하버
            ...     
2045      플레이스 캠프 제주
2046      플레이스 캠프 제주
2047      플레이스 캠프 제주
2048      플레이스 캠프 제주
2049      플레이스 캠프 제주
Name: hotel, Length: 2050, dtype: object

In [23]:
for i, doc in enumerate(df_1_['review']):
    # # 문서를 추가
    # if i >= len(df_1_):
    #     break  # DataFrame 범위를 벗어나면 루프 종료
    embedding = model.encode(doc).tolist()
    hotel_collection.add(
        ids=[str(i)],
        embeddings=embedding,
        metadatas = [{'hotel': df_1_['hotel'][i], "review": doc}]
    )  

In [24]:
print("현재 저장된 문서 개수:", hotel_collection.count())


현재 저장된 문서 개수: 2050


In [48]:
query_keyword = "가격대비 좋은 호텔"


query_embedding = model.encode(query_keyword).tolist()

results = hotel_collection.query(
    query_embeddings=query_embedding,
    n_results=3
)

for i, result in enumerate(results["metadatas"][0]):
    print("추천 호텔:", result["hotel"])
    print("추천 호텔 리뷰:", result["review"])
    print("="*100)

추천 호텔: 호텔 에어시티 제주
추천 호텔 리뷰: 가성비 좋은 호텔이네요. 잘 쉬다갑니다.
추천 호텔: 호텔 에어시티 제주
추천 호텔 리뷰: 가성비 짱좋은 호텔. 위치도 굳 주차가 조금번거로웠지만 매우만족합니다^^
추천 호텔: 코업시티호텔 성산
추천 호텔 리뷰: 방 컨디션 및 뷰가 너무 좋구요!
조식도 저렴한데 엄청맛잇고 만족스러웟어요!
성산일출봉도 가까워서 일출보기도 너무좋은 최적의 호텔이엿어요!!!
